In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Mouse

## Quake_10x_Liver

In [ ]:
quake_10x_liver = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_10x_Liver/data.h5")
utils.peek(quake_10x_liver, "build/liver/Quake_10x_Liver")
quake_10x_liver.obs.head()

In [ ]:
quake_10x_liver.obs["cluster"] = "cluster_" + quake_10x_liver.obs["cluster"].astype(int).astype(str)
quake_10x_liver.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor", h_dim=32, batch_size=64)
quake_10x_liver_model = cb.directi.fit_DIRECTi(
    quake_10x_liver, quake_10x_liver.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
quake_10x_liver.latent = quake_10x_liver_model.inference(quake_10x_liver)

In [ ]:
ax = quake_10x_liver.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/Quake_10x_Liver/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_liver.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/Quake_10x_Liver/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_liver.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/Quake_10x_Liver/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_liver.visualize_latent("free_annotation", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/Quake_10x_Liver/free_annotation.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_liver.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/Quake_10x_Liver/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_10x_liver.write_dataset("build/liver/Quake_10x_Liver/Quake_10x_Liver.h5")

In [ ]:
%%capture capio
quake_10x_liver_models = [quake_10x_liver_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_10x_liver_models.append(cb.directi.fit_DIRECTi(
        quake_10x_liver, quake_10x_liver.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_10x_liver_blast = cb.blast.BLAST(quake_10x_liver_models, quake_10x_liver)
quake_10x_liver_blast.save("build/liver/Quake_10x_Liver")

In [ ]:
with open("build/liver/Quake_10x_Liver/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/liver/Quake_10x_Liver/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_10x_liver_blast, "build/liver/Quake_10x_Liver")

In [ ]:
%%writefile build/liver/Quake_10x_Liver/predictable.txt
cell_ontology_class
cell_type1
cluster
free_annotation

## Mouse aligned

In [ ]:
if not os.path.exists("build/liver/ALIGNED_Mus_musculus_Liver"):
    os.makedirs("build/liver/ALIGNED_Mus_musculus_Liver")
quake_smart_seq2_liver = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Liver/data.h5")
quake_smart_seq2_liver.obs["cluster"] = "cluster_" + quake_smart_seq2_liver.obs["cluster"].astype(int).astype(str)
yang = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Yang/data.h5")
yang.obs["batch"] = "batch_" + yang.obs["batch"].astype(int).astype(str)
quake_10x_liver.obs["quake_10x_liver_donor"] = quake_10x_liver.obs["donor"]
quake_smart_seq2_liver.obs["quake_smart_seq2_liver_donor"] = quake_smart_seq2_liver.obs["donor"]
mouse = cb.data.ExprDataSet.merge_datasets(dict(
    quake_10x_liver=quake_10x_liver, quake_smart_seq2_liver=quake_smart_seq2_liver, yang=yang
), merge_uns_slots=["seurat_genes"])

In [ ]:
mouse.obs.dtypes

In [ ]:
opt_model_kwargs = dict(
    batch_effect=["dataset_name", "quake_10x_liver_donor", "quake_smart_seq2_liver_donor"],
    rmbatch_module_kwargs=[dict(lambda_reg=0.005), dict(lambda_reg=0.01), dict(lambda_reg=0.01)],
    batch_size=64
)
mouse_model = cb.directi.fit_DIRECTi(
    mouse, mouse.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
mouse.latent = mouse_model.inference(mouse)

In [ ]:
ax = mouse.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/ALIGNED_Mus_musculus_Liver/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/ALIGNED_Mus_musculus_Liver/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
mouse.write_dataset("build/liver/ALIGNED_Mus_musculus_Liver/ALIGNED_Mus_musculus_Liver.h5")

In [ ]:
%%capture capio
mouse_models = [mouse_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    mouse_models.append(cb.directi.fit_DIRECTi(
        mouse, mouse.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
mouse_blast = cb.blast.BLAST(mouse_models, mouse)
mouse_blast.save("build/liver/ALIGNED_Mus_musculus_Liver")

In [ ]:
with open("build/liver/ALIGNED_Mus_musculus_Liver/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/liver/ALIGNED_Mus_musculus_Liver/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(mouse_blast, "build/liver/ALIGNED_Mus_musculus_Liver")

In [ ]:
%%writefile build/liver/ALIGNED_Mus_musculus_Liver/predictable.txt
cell_ontology_class

---

# Human

## MacParland

In [ ]:
macparland = cb.data.ExprDataSet.read_dataset("../../Datasets/data/MacParland/data.h5")
utils.peek(macparland, "build/liver/MacParland")
macparland.obs.head()

In [ ]:
macparland.obs.dtypes

In [ ]:
utils.check_meta(macparland.obs)

In [ ]:
macparland.obs.dtypes

In [ ]:
macparland.uns["scanpy_genes"].shape

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
macparland_model = cb.directi.fit_DIRECTi(
    macparland, macparland.uns["scanpy_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
macparland.latent = macparland_model.inference(macparland)

In [ ]:
# 1435 genes
ax = macparland.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macparland.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macparland.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macparland.visualize_latent("cell_cycle", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/cell_cycle.svg", dpi=utils.DPI, bbox_inches="tight")


In [ ]:
ax = macparland.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macparland.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macparland.visualize_latent("race", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/liver/MacParland/race.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
macparland.write_dataset("build/liver/MacParland/MacParland.h5")

In [ ]:
%%capture capio
macparland_models = [macparland_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    macparland_models.append(cb.directi.fit_DIRECTi(
        macparland, macparland.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
macparland_blast = cb.blast.BLAST(macparland_models, macparland)
macparland_blast.save("build/liver/MacParland")

In [ ]:
with open("build/liver/MacParland/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/liver/MacParland/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(macparland_blast, "build/liver/MacParland")

In [ ]:
%%writefile build/liver/MacParland/predictable.txt
cell_ontology_class
cell_type1
cluster